In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (10,10)
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
df = pd.read_csv('Training Data.csv')
df.rename(columns={"Married/Single":"Married_Single"}, inplace=True)

In [2]:
encoder = LabelEncoder()
df["Married_Single"]=encoder.fit_transform(df["Married_Single"])
df["Car_Ownership"]= encoder.fit_transform(df["Car_Ownership"])
df = df.drop(['Id' ,'Profession' , 'CITY' , 'STATE' , 'House_Ownership'] ,axis = 1)

In [3]:
df.head()

,Income,Age,Experience,Married_Single,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1303834,23,3,1,0,3,13,0
1,7574516,40,10,1,0,9,13,0
2,3991815,66,4,0,0,4,10,0
3,6256451,41,2,1,1,2,12,1
4,5768871,47,11,1,0,3,14,1


In [4]:
df.describe()

,Income,Age,Experience,Married_Single,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
count,2.520000e+05,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000
mean,4.997117e+06,49.954071,10.084437,0.897905,0.301587,6.333877,11.997794,0.123000
std,2.878311e+06,17.063855,6.002590,0.302774,0.458948,3.647053,1.399037,0.328438
min,1.031000e+04,21.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000
25%,2.503015e+06,35.000000,5.000000,1.000000,0.000000,3.000000,11.000000,0.000000
50%,5.000694e+06,50.000000,10.000000,1.000000,0.000000,6.000000,12.000000,0.000000
75%,7.477502e+06,65.000000,15.000000,1.000000,1.000000,9.000000,13.000000,0.000000
max,9.999938e+06,79.000000,20.000000,1.000000,1.000000,14.000000,14.000000,1.000000


In [7]:
max(df.Age) , min(df.Age)

(79, 21)

In [5]:
max(df.Income) , min(df.Income)

(9999938, 10310)

In [8]:
max(df.Experience) , min(df.Experience)

(20, 0)

In [9]:
max(df.CURRENT_JOB_YRS) , min(df.CURRENT_JOB_YRS)

(14, 0)

In [10]:
max(df.CURRENT_HOUSE_YRS) , min(df.CURRENT_HOUSE_YRS)

(14, 10)

In [12]:
X =df.drop("Risk_Flag" , axis = 1)
y = df["Risk_Flag"]
smote = SMOTE(sampling_strategy='minority')
X_sm_original , y_sm_original  = smote.fit_resample(X , y)

In [13]:
X.head()

,Income,Age,Experience,Married_Single,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,1303834,23,3,1,0,3,13
1,7574516,40,10,1,0,9,13
2,3991815,66,4,0,0,4,10
3,6256451,41,2,1,1,2,12
4,5768871,47,11,1,0,3,14


In [28]:
X_sm_original[:1]

,Income,Age,Experience,Married_Single,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,1303834,23,3,1,0,3,13


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

sc =MinMaxScaler()
X_sm = sc.fit_transform(X_sm_original)

In [19]:
sc.fit(X_sm_original , y_sm_original)

MinMaxScaler()

In [20]:
sc.transform([[1303834,23,3,1,0 ,3 ,13]])

array([[0.1294867 , 0.03448276, 0.15      , 1.        , 0.        ,
        0.21428571, 0.75      ]])

In [21]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_sm , y_sm_original)

RandomForestClassifier(n_estimators=20)

In [26]:
X_sm[:1] , y_sm_original[:1]

(array([[0.1294867 , 0.03448276, 0.15      , 1.        , 0.        ,
         0.21428571, 0.75      ]]),
 0    0
 Name: Risk_Flag, dtype: int64)

In [23]:
import pickle 
pickle_out = open("classifier_new.pkl" , "wb")
pickle.dump(model , pickle_out)
pickle_out.close()

In [24]:
pickle_in = open('classifier_new.pkl' ,'rb')
classifier = pickle.load(pickle_in)

In [27]:
classifier.predict([[0.1294867 , 0.03448276, 0.15      , 1.        , 0.        ,
        0.21428571, 0.75    ]])

array([0], dtype=int64)

In [32]:
classifier.predict_proba([[-1.27213597, -1.56631484, -1.14406481,  0.34166025, -0.60443274,
        -0.8853698 ,  0.76312142]])

array([[0.65, 0.35]])